<a href="https://colab.research.google.com/github/nidjosep/student-feedback-analysis/blob/master/models/model-source/Model_2_Actionable_Insights_from_Student_Feedback_SA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import ssl
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive', force_remount=True)

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/MCSC/TME_6015/Project/Emotion_final.csv')

Mounted at /content/drive


In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout, Bidirectional
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd


texts = df['Review']
labels = df['Label']

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and return encoded labels
encoded_labels = label_encoder.fit_transform(labels)

# To see the mapping of string labels to integers
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping: ", label_mapping)

# Hyperparameters
vocab_size = 10000  # Number of unique words in the vocabulary
embedding_dim = 100  # Number of dimensions for each word vector
max_length = 100    # Max length of each input sequence
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"   # Token for out-of-vocabulary words

# Tokenize the text
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(padded, encoded_labels, test_size=0.2, random_state=42)

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(GRU(64, return_sequences=True)),
    Bidirectional(GRU(32)),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

# Using Adam optimizer with a different learning rate
optimizer = Adam(learning_rate=0.0005)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=True)

history = model.fit(train_texts, train_labels, epochs=30, batch_size=64, validation_data=(val_texts, val_labels), callbacks=[early_stopping])

Label Mapping:  {'anger': 0, 'fear': 1, 'happy': 2, 'love': 3, 'sadness': 4, 'surprise': 5}
Epoch 1/30
269/269 [==============================] - 22s 60ms/step - loss: 1.8366 - accuracy: 0.3604 - val_loss: 1.3454 - val_accuracy: 0.5128
Epoch 2/30
269/269 [==============================] - 7s 25ms/step - loss: 1.0258 - accuracy: 0.6402 - val_loss: 0.7885 - val_accuracy: 0.7300
Epoch 3/30
269/269 [==============================] - 6s 21ms/step - loss: 0.6459 - accuracy: 0.7804 - val_loss: 0.6172 - val_accuracy: 0.8059
Epoch 4/30
269/269 [==============================] - 5s 20ms/step - loss: 0.4172 - accuracy: 0.8838 - val_loss: 0.5195 - val_accuracy: 0.8465
Epoch 5/30
269/269 [==============================] - 5s 20ms/step - loss: 0.3179 - accuracy: 0.9215 - val_loss: 0.4875 - val_accuracy: 0.8702
Epoch 6/30
269/269 [==============================] - 5s 19ms/step - loss: 0.2616 - accuracy: 0.9401 - val_loss: 0.4717 - val_accuracy: 0.8719
Epoch 7/30
269/269 [=============================

In [23]:
# predict with one sample
import json

def predict_sentiment(text):
    # Tokenize and pad the text
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    # Predict
    prediction = model.predict(padded_sequence)
    sentiment = label_encoder.inverse_transform([np.argmax(prediction)])[0]

    return sentiment

predict_sentiment("The practical examples aspect of the course was engaging, but the practical examples could use more attention.")

1/1 [==============================] - 0s 22ms/step


'happy'

In [18]:
# predict with test dataset

def read_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

test_data_dict = read_json('/content/drive/MyDrive/MCSC/TME_6015/Project/test_dataset_all_models.json')

test_messages = list(test_data_dict.values())

predicted_sentiments = {str(i+1): predict_sentiment(message) for i, message in enumerate(test_messages)}

# Write the dictionary to a JSON file
with open('/content/drive/MyDrive/MCSC/TME_6015/Project/emotions.json', 'w') as json_file:
    json.dump(predicted_sentiments, json_file, indent=4)

print("Sentiments written to emotions.json")

1/1 [==============================] - 0s 21ms/step
Sentiments written to emotions.json


In [19]:
import json

# Predict sentiments for all messages
predicted_sentiments = {str(i+1): predict_sentiment(message) for i, message in enumerate(test_messages)}

# Write the dictionary to a JSON file
with open('/content/drive/MyDrive/MCSC/TME_6015/Project/emotions.json', 'w') as json_file:
    json.dump(predicted_sentiments, json_file, indent=4)

print("Sentiments written to emotions.json")

1/1 [==============================] - 0s 22ms/step
Sentiments written to emotions.json
